In [2]:
import datetime

import numpy as np
import pandas as pd
import yaml
from sqlalchemy import create_engine


# database connections 

In [3]:
with open('config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_co = config['CO_SA']
    config_etl = config['ETL_PRO']

# Construct the database URL
url_co = (f"{config_co['drivername']}://{config_co['user']}:{config_co['password']}@{config_co['host']}:"
          f"{config_co['port']}/{config_co['dbname']}")
url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:"
           f"{config_etl['port']}/{config_etl['dbname']}")
# Create the SQLAlchemy Engine
co_sa = create_engine(url_co)
etl_conn = create_engine(url_etl)

# Extract

In [4]:
df_trans = pd.read_sql_table('trans_servicio', etl_conn)
dim_persona = pd.read_sql_table('dim_persona', etl_conn)
dim_medico = pd.read_sql_table('dim_medico', etl_conn)
dim_servicio = pd.read_sql_table('dim_servicio', etl_conn)
dim_ips = pd.read_sql_table('dim_ips', etl_conn)
dim_fecha = pd.read_sql_table('dim_fecha', etl_conn)

# Transformations

In [11]:
hecho_atencion = pd.merge(df_trans,dim_fecha[['date','key_fecha']],left_on='fecha_atencion',right_on='date')
hecho_atencion.drop(columns=['date'], inplace=True)
hecho_atencion.rename(columns={'key_fecha':'key_fecha_atencion','id_medico':'cedula','id_usuario':'numero_identificacion'}, inplace=True)
hecho_atencion = pd.merge(hecho_atencion,dim_fecha[['date','key_fecha']],left_on='fecha_solicitud',right_on='date')
hecho_atencion.drop(columns=['date'], inplace=True)
hecho_atencion.rename(columns={'key_fecha':'key_fecha_solicitud'}, inplace=True)

hecho_atencion.head()

,key_trans,codigo_servicio,numero_identificacion,cedula,fecha_solicitud,hora_solicitud,fecha_atencion,hora_atencion,tipo_servicio,fecha_hora_atencion,fecha_hora_solicitud,key_fecha_atencion,key_fecha_solicitud,key_persona
0,0,808809,1705225684180,272113543,2007-03-19,28800000000000,2007-03-19,30600000000000,hospitalizacion,2007-03-19 08:30:00,2007-03-19 08:00:00,807,807,2841
1,1,808811,1705225699140,81113543,2006-12-25,28800000000000,2006-12-25,30600000000000,hospitalizacion,2006-12-25 08:30:00,2006-12-25 08:00:00,723,723,2846
2,2,808813,1705225788780,166113543,2007-12-16,28800000000000,2007-12-16,30480000000000,hospitalizacion,2007-12-16 08:28:00,2007-12-16 08:00:00,1079,1079,2899
3,3,808815,1705225786530,228113543,2007-01-07,28800000000000,2007-01-07,30840000000000,hospitalizacion,2007-01-07 08:34:00,2007-01-07 08:00:00,736,736,2897
4,4,808817,1705225880540,172113543,2006-10-29,28800000000000,2006-10-29,30360000000000,hospitalizacion,2006-10-29 08:26:00,2006-10-29 08:00:00,666,666,2957


In [17]:
hecho_atencion= hecho_atencion.merge(dim_persona[['key_persona','numero_identificacion']])
hecho_atencion.drop(columns=['numero_identificacion'], inplace=True)

hecho_atencion= hecho_atencion.merge(dim_medico[['key_medico','cedula','id_ips']])
hecho_atencion.drop(columns=['cedula'], inplace=True)

hecho_atencion= hecho_atencion.merge(dim_ips[['key_ips','id_ips']])
hecho_atencion.drop(columns=['id_ips'], inplace=True)
hecho_atencion.head()

,key_trans,codigo_servicio,fecha_solicitud,hora_solicitud,fecha_atencion,hora_atencion,fecha_hora_atencion,fecha_hora_solicitud,key_fecha_atencion,key_fecha_solicitud,key_persona,key_medico,key_ips,key_servicio
0,0,808809,2007-03-19,28800000000000,2007-03-19,30600000000000,2007-03-19 08:30:00,2007-03-19 08:00:00,807,807,2841,271,23,1
1,1,808811,2006-12-25,28800000000000,2006-12-25,30600000000000,2006-12-25 08:30:00,2006-12-25 08:00:00,723,723,2846,80,5,1
2,2,808813,2007-12-16,28800000000000,2007-12-16,30480000000000,2007-12-16 08:28:00,2007-12-16 08:00:00,1079,1079,2899,165,13,1
3,3,808815,2007-01-07,28800000000000,2007-01-07,30840000000000,2007-01-07 08:34:00,2007-01-07 08:00:00,736,736,2897,227,17,1
4,4,808817,2006-10-29,28800000000000,2006-10-29,30360000000000,2006-10-29 08:26:00,2006-10-29 08:00:00,666,666,2957,171,14,1


In [16]:
hecho_atencion= hecho_atencion.merge(dim_servicio[['name','key_servicio']],left_on='tipo_servicio',right_on='name')
hecho_atencion.drop(columns=['name','tipo_servicio'], inplace=True)
hecho_atencion.head()

,key_trans,codigo_servicio,fecha_solicitud,hora_solicitud,fecha_atencion,hora_atencion,fecha_hora_atencion,fecha_hora_solicitud,key_fecha_atencion,key_fecha_solicitud,key_persona,key_medico,key_ips,key_servicio
0,0,808809,2007-03-19,28800000000000,2007-03-19,30600000000000,2007-03-19 08:30:00,2007-03-19 08:00:00,807,807,2841,271,23,1
1,1,808811,2006-12-25,28800000000000,2006-12-25,30600000000000,2006-12-25 08:30:00,2006-12-25 08:00:00,723,723,2846,80,5,1
2,2,808813,2007-12-16,28800000000000,2007-12-16,30480000000000,2007-12-16 08:28:00,2007-12-16 08:00:00,1079,1079,2899,165,13,1
3,3,808815,2007-01-07,28800000000000,2007-01-07,30840000000000,2007-01-07 08:34:00,2007-01-07 08:00:00,736,736,2897,227,17,1
4,4,808817,2006-10-29,28800000000000,2006-10-29,30360000000000,2006-10-29 08:26:00,2006-10-29 08:00:00,666,666,2957,171,14,1


# tiempo de espera

In [31]:
hecho_atencion['tiempo_espera'] = hecho_atencion['fecha_hora_atencion'] - hecho_atencion['fecha_hora_solicitud']
hecho_atencion['tiempo_espera_dias'] = hecho_atencion['tiempo_espera'].dt.day
#hecho_atencion['tiempo_espera_minutos'] = hecho_atencion['tiempo_espera'].dt.minute
#hecho_atencion['tiempo_espera_horas'] = hecho_atencion['tiempo_espera'].dt.hour
#hecho_atencion['tiempo_espera_segundos'] = hecho_atencion['tiempo_espera'].dt.second
hecho_atencion.head()


AttributeError: 'TimedeltaProperties' object has no attribute 'day'

# load

In [ ]:
trans_servicio.to_sql('trans_servicio',etl_conn,if_exists='replace',index_label='key_trans')